### Set user-defined parameters

In [ ]:
### Set user-defined parameters for analysis ###

# Point the settings file to JSON file with analysis parameters.
# This path (and others in this notebook) can be relative or absolute.
#settings_file = '../data/analysis_settings/2021_12_03_reviewerExpts.json'
settings_file = '../data/analysis_settings/FigXX_HaloTagControls_Diffusion_test.json'

# Set plotting and figure saving params
plot_figs = True
plot_all_tracks = False # warning - takes forever!
save_figs = True # Save output figures
save_data = True # Save filtered tracks to see which ones were identified as correlated.

plot_settings = '../src/plotting_settings.py' # File containing matplotlib settings
save_dir_reports = '../reports/figures' # Base directory for saving figures

### Load modules and data

In [ ]:
### Load modules ###

# Uncomment the following two lines for debugging
%load_ext autoreload
%autoreload 2

# Import general Python modules
import os, sys, inspect
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns

# Add source code directory (src) to path to enable user module import
module_dir = '../src'
os.sys.path.insert(0, module_dir)

# Import user modules from source code directory
import parse_trackmate as pt
import correlation_analysis as corr
import diffusion as dif

In [ ]:
### Load the JSON settings file that specifies analysis details ###
# Note: for diffusion analysis, the JSON settings file must contain
# the following keys: 'diff_dim', 'min_averages_for_msd', 'dc_fit_nframes',
# and "frame_interval_sec" (the latter is in seconds)

conditions, params = corr.read_analysis_params(settings_file, 
                                               save_dir_reports, 
                                               print_summary=True)

# Parse the analysis settings specific to MSD analysis
diff_dim = params['raw_json']['diff_dim']
min_averages_for_msd = params['raw_json']['min_averages_for_msd']
dc_fit_nframes = params['raw_json']['dc_fit_nframes']
frame_interval = params['raw_json']['frame_interval_sec']

print(diff_dim, min_averages_for_msd, dc_fit_nframes, frame_interval)

In [ ]:
### Set up figure save directories and load plotting style ###

save_dir = params['save_dir']
save_dir_data = params['save_dir_data']

if save_figs: # Save figure files
    %matplotlib
    %run $plot_settings save_large
    
    # Make directories for saving figures
    dir_sum_figs = os.path.join(save_dir, 'summary_figures')
    dir_examples = os.path.join(save_dir, 'examples') 
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    if not os.path.exists(dir_sum_figs):
        os.makedirs(dir_sum_figs)
    if not os.path.exists(dir_examples):
        os.makedirs(dir_examples)    
else: # Plot interactively
    %matplotlib
    %run $plot_settings plot_only
    
if save_data: # Save filtered TrackMate trajectories
    if not os.path.exists(save_dir_data):
        os.makedirs(save_dir_data)

In [ ]:
### Parse track data from TrackMate xml files into Pandas dataframes ###

data_parsed = {} # List of dataframes, one dataframe per condition
for condition in conditions:
    print("Now processing condition: " + condition)
    data_parsed[condition] = pt.read_2color_data(conditions[condition],
                                    do_int_analysis=params['do_int_analysis'],
                                    int_settings=params['int_settings'])
print('Done parsing. Data loading is now complete.')

## Function definitions for MSD analysis

In [ ]:
# Defines a function that can take in the movies and return track trajectories in the form of coordinates within
# pandas dataframe corresponding to each channel.

def get_movie_data_both_channels(data, movie_ID):
    print("Processing movie:", movie_ID)
    
    # Get indices of the movies of both colors, denoted by C1 and C2
    movie_idx = data['movie_ID'] == movie_ID
    idx_C1 = data.index[movie_idx & (data['color'] == 'C1')]
    idx_C2 = data.index[movie_idx & (data['color'] == 'C2')]
    
    # Locate data for each color of the current movie
    df_C1 = data.loc[idx_C1, 'parsed'].iat[0]
    df_C2 = data.loc[idx_C2, 'parsed'].iat[0]
    
    # Locate corresponding file names and paths
    filename_C1 = data.loc[idx_C1, 'file_name'].iat[0]
    file_path_C1 = data.loc[idx_C1, 'file_path'].iat[0]
    filename_C2 = data.loc[idx_C2, 'file_name'].iat[0]
    file_path_C2 = data.loc[idx_C2, 'file_path'].iat[0]
    
    names_paths = {'name_C1' : filename_C1, 'path_C1' : file_path_C1,
                    'name_C2' : filename_C1, 'path_C2' : file_path_C2}
    
    return df_C1, df_C2, names_paths

# Define a function to pool the tracks in a channel

def pool_tracks(channel_dat):
    channel_tracks = []
    cols = ['x', 'y', 't']
    for i in range(0, max(channel_dat['track_ID'])):
        track = np.array(channel_dat[channel_dat['track_ID'] == i][cols])
        channel_tracks.append(track)
    return channel_tracks

# Define function to calculate MSD for each track and fit to a
# diffusion coefficient.

def calculate_msd(tracks, frame_interval):
    
    d = []
    for track in tracks:
        len_track = np.shape(track)[0]
        if len_track < (dc_fit_nframes * min_averages_for_msd):
            continue

        # Break up the track into sub-trajectories for MSD calc'n
        track_chunks = []
        n_chunks = int(np.floor(len_track / dc_fit_nframes))
        for i in range(n_chunks):
            start_slice = i * dc_fit_nframes
            end_slice = start_slice + dc_fit_nframes
            chunk = track[start_slice:end_slice, :]
            track_chunks.append(chunk)

        # Get the data
        t_dsq, msd_data = dif.calc_msd(track_chunks, frame_interval)

        time, mean_dsq, std_dsq, sterr_dsq = msd_data

        # Fit to get the diffusion coefficient
        fit_params = dif.fit_diffusion_const(msd_data, dim = diff_dim,
                                            nframes = dc_fit_nframes)

        d.append({'Track_ID':i, 'Diff_const':fit_params['dc']})


    data = pd.DataFrame(d)
    
    return data


In [ ]:
# Define a function for calculating the MSD for particles in 
# each channel for each movie in each condition

def get_condition_msd(all_data, condition):
    data = all_data[condition]
    
    condition_msd = []
    movie_counter = 1
    for movie in data:
        print('Now processing movie:', str(movie_counter))
        movie_counter += 1
        movie_msd = []
        channel_counter = 1
        for channel in movie:
            print('Now processing channel:', str(channel_counter))
            channel_counter += 1
            channel_tracks = pool_tracks(channel)
            msd_dat = calculate_msd(channel_tracks, frame_interval)
            movie_msd.append(msd_dat)
        condition_msd.append(movie_msd)
    print('Processing finished!')
    return condition_msd

## Data formatting and MSD calculation

In [ ]:
# Format the data in an accessible manner - this will be a dictionary that has the condition as keys. The 
# values are lists that contains coordinate information for each channel in the movie.
all_dat = dict()
for condition in conditions:
    all_dat[condition] = list()
    condition_dat = data_parsed[condition]
    for movie_id in condition_dat['movie_ID'].unique():
        df_C1, df_C2, names_paths = get_movie_data_both_channels(condition_dat,
                                                                movie_id)
        all_dat[condition].append([df_C1, df_C2])
        

In [ ]:
# Similar to the code block above, this code block will give you a dictionary that uses the condition as the keys.
# The values will be a list containing the MSDs of particles associated with a treatment condition. This code block
# takes a while, especially if you are pooling datasets.

condition_msds = dict()
for condition in conditions:
    print('Now processing:', condition)
    condition_msds[condition] = get_condition_msd(all_dat, condition)
print('Really finished')

In [ ]:
# Appends the "condition" column to the current dataframe to allow for subsequent subsetting/slicing of data
def append_condition(condition_channel_df, condition, channel):
    condition_list = list()
    channel_list = list()
    for i in range(0, len(condition_channel_df)):
        condition_list.append(condition)
        if channel == 0:
            channel_list.append('Green')
        elif channel == 1:
            channel_list.append('Red')
    condition_channel_df['Condition'] = condition_list
    condition_channel_df['Channel'] = channel_list
    return condition_channel_df

## Data formatting

In [ ]:
# Code block for making a Pandas dataframe suitable for plotting violin plots.
master_df = pd.DataFrame()
for condition in conditions:
    for movie in condition_msds[condition]:
        for channel in range(0, len(movie)):
            master_df = master_df.append(append_condition(movie[channel], 
                                                          condition, 
                                                          channel))

In [ ]:
# Optional code block for seeing the MSDs in log10 scale
log_dc = list()
dc_10 = list()
for index, row in master_df.iterrows():
    log_dc.append(np.log10(row['Diff_const']))
    dc_10.append(np.power(10, row['Diff_const']))
master_df['log10_DC'] = log_dc
master_df['DC_10'] = dc_10

## Data Visualization

In [ ]:
# Create a violin plot that splits into red or green channels
ax = sns.violinplot(x = 'Condition', 
                    y = 'log10_DC', data = master_df,
                    hue = master_df.Channel, split = True, cut = 0,
                   palette=['green', 'red'])
ax.set_title('Diffusion Constants of IRE1 Molecules in CL-VB-69.4 Cells')
ax.set_ylabel('Diffusion Constant')
ax.set_xlabel('Conditions')
ax.axhline(0.1, linestyle = '--', color = 'black')
#ax.set_yscale("log")

plt.show()

In [ ]:
# Differences across conditions, regardless of spot channel

fig2 = plt.figure()

ax = sns.violinplot(x = 'Condition', 
                    y = 'log10_DC', data = master_df, cut = 0, color = 'steelblue')
ax.set_title('MSD of Correlated Tracks in Green Channel')
ax.set_ylabel('Diffusion Constant')
ax.set_xlabel('Conditions')
ax.axhline(0.1, linestyle = '--', color = 'black')
#ax.set_yscale("log")


plt.show()

### To-do: write a script somewhere that can output filtered TrackMate files depending on some minimal or maximal value of MSD.